In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer
import pandas as pd
from sklearn.metrics import f1_score, classification_report
import os

# 커스텀 모델 로드
from model_training import MultiLabelClassifier

# 모델과 토크나이저 로드
model_path = r"C:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\text-model\models\bert_model"
checkpoint = torch.load(os.path.join(model_path, "model.pt"))
model = MultiLabelClassifier()
model.load_state_dict(checkpoint['model_state_dict'])
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 테스트 데이터 로드
data_path = r"C:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\text-model\data\processed_data.csv"
data = pd.read_csv(data_path)

# 입력 문장 토크나이징
inputs = tokenizer(list(data['입력 문장']), padding=True, truncation=True, return_tensors="pt")

# 레이블 매핑
도수_매핑 = {'낮은': 0, '중간': 1, '높은': 2}
술종류_매핑 = {'칵테일': 0, '럼': 1, '위스키': 2, '보드카': 3}
맛_매핑 = {'달달한': 0, '쓴맛': 1, '상큼한': 2, '신맛': 3, '부드러운': 4}

# 레이블 변환
도수_labels = torch.tensor([도수_매핑[도수] for 도수 in data['도수']])
술종류_labels = torch.tensor([술종류_매핑[종류] for 종류 in data['술 종류']])
맛_labels = torch.tensor([맛_매핑[맛] for 맛 in data['맛']])

# 데이터셋 생성
dataset = TensorDataset(
    inputs['input_ids'], 
    inputs['attention_mask'], 
    도수_labels,
    술종류_labels,
    맛_labels
)
dataloader = DataLoader(dataset, batch_size=2)

# 모델 평가
model.eval()
all_preds_도수 = []
all_preds_술종류 = []
all_preds_맛 = []
all_labels_도수 = []
all_labels_술종류 = []
all_labels_맛 = []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, 도수_label, 술종류_label, 맛_label = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # 각 분류기의 예측값 계산
        도수_pred = torch.argmax(outputs['도수'], dim=1)
        술종류_pred = torch.argmax(outputs['술종류'], dim=1)
        맛_pred = torch.argmax(outputs['맛'], dim=1)
        
        # 예측값과 실제값 저장
        all_preds_도수.extend(도수_pred.cpu().numpy())
        all_preds_술종류.extend(술종류_pred.cpu().numpy())
        all_preds_맛.extend(맛_pred.cpu().numpy())
        all_labels_도수.extend(도수_label.cpu().numpy())
        all_labels_술종류.extend(술종류_label.cpu().numpy())
        all_labels_맛.extend(맛_label.cpu().numpy())

# 각 분류기별 성능 평가
print("도수 분류 결과:")
print(classification_report(all_labels_도수, all_preds_도수, 
                          target_names=['낮은', '중간', '높은']))

print("\n술종류 분류 결과:")
print(classification_report(all_labels_술종류, all_preds_술종류, 
                          target_names=['칵테일', '럼', '위스키', '보드카']))

print("\n맛 분류 결과:")
print(classification_report(all_labels_맛, all_preds_맛, 
                          target_names=['달달한', '쓴맛', '상큼한', '신맛', '부드러운']))

C:\Users\user\AppData\Local\Temp\ipykernel_8472\1390757066.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(model_path, "model.pt"))

도수 분류 결과:
              precision    recall  f1-score   support

          낮은       0.00      0.00      0.00         8
          중간       0.00      0.00      0.00         6
          높은       0.44      1.00      0.61        11

    accuracy                           0.44        25
   macro avg       0.15      0.33      0.20        25
weighted avg       0.19      0.44      0.27        25


술종류 분류 결과:
              precision    recall  f1-score   support

         칵테일       0.40      1.00      0.57        10
           럼       0.00      0.00      0.00         5
         위스키       0.00      0.00      0.00         5
         보드카       0.00      0.00      0.00         5

    accuracy                           0.40        25
   macro avg       0.10      0.25      0.14        25
weighted avg       0.16      0.40      0.23        25


맛 분류 결과:
              precision    recall  f1-score   support

         달달한       0.28      1.00      0.44         7
          쓴맛       0.00      0.00      0.00

c:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av